К сожалению, проблемы установки СУБД PostgreSQL на моем компьютере пока не решены, поэтому, как и с прошлым ДЗ, выполняю задание в Google Colab с имитацией СУБД.

In [32]:
import duckdb
import pandas as pd

con = duckdb.connect(database=':memory:')

con.execute("""
CREATE TABLE customer AS SELECT * FROM read_csv_auto('customer.csv', delim=';');
CREATE TABLE order_items AS SELECT * FROM read_csv_auto('order_items.csv');
CREATE TABLE orders AS SELECT * FROM read_csv_auto('orders.csv');
CREATE TABLE product AS SELECT * FROM read_csv_auto('product.csv');
""")

1.	Вывести все уникальные бренды, у которых есть хотя бы один продукт со стандартной стоимостью выше 1500 долларов, и суммарными продажами не менее 1000 единиц.

In [33]:
query_1 = """
SELECT DISTINCT p.brand
FROM product p
JOIN order_items oi ON p.product_id = oi.product_id
WHERE p.standard_cost > 1500
GROUP BY p.brand
HAVING SUM(oi.quantity) >= 1000
"""

result_1 = pd.read_sql(query_1, con)
result_1

/tmp/ipython-input-2405624950.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_1 = pd.read_sql(query_1, con)


,brand
0,OHM Cycles
1,Giant Bicycles
2,Trek Bicycles
3,Solex


2.	Для каждого дня в диапазоне с 2017-04-01 по 2017-04-09 включительно вывести количество подтвержденных онлайн-заказов и количество уникальных клиентов, совершивших эти заказы.

In [34]:
query_2 = """
SELECT
    order_date,
    COUNT(order_id) AS confirmed_online_orders,
    COUNT(DISTINCT customer_id) AS unique_customers
FROM orders
WHERE
    online_order = TRUE
    AND order_status = 'Approved'
    AND order_date BETWEEN DATE '2017-04-01' AND DATE '2017-04-09'
GROUP BY order_date
ORDER BY order_date
"""

result_2 = pd.read_sql(query_2, con)
result_2


/tmp/ipython-input-3312792767.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_2 = pd.read_sql(query_2, con)


,order_date,confirmed_online_orders,unique_customers
0,2017-04-01,37,37
1,2017-04-02,29,29
2,2017-04-03,27,27
3,2017-04-04,32,32
4,2017-04-05,33,32
5,2017-04-06,36,36
6,2017-04-07,24,24
7,2017-04-08,33,33
8,2017-04-09,30,30


3.	Вывести профессии клиентов:

	из сферы IT, чья профессия начинается с Senior;

	из сферы Financial Services, чья профессия начинается с Lead.
     
Для обеих групп учитывать только клиентов старше 35 лет. Объединить выборки с помощью UNION ALL.


In [35]:
query_3 = """
SELECT job_title, job_industry_category, DOB
FROM customer
WHERE job_industry_category = 'IT'
  AND job_title LIKE 'Senior%'
  AND DATE_PART('year', CURRENT_DATE) - DATE_PART('year', DOB) > 35

UNION ALL

SELECT job_title, job_industry_category, DOB
FROM customer
WHERE job_industry_category = 'Financial Services'
  AND job_title LIKE 'Lead%'
  AND DATE_PART('year', CURRENT_DATE) - DATE_PART('year', DOB) > 35
"""

result_3 = pd.read_sql(query_3, con)
result_3

/tmp/ipython-input-1079737710.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_3 = pd.read_sql(query_3, con)


,job_title,job_industry_category,DOB
0,Senior Sales Associate,IT,1976-11-29
1,Senior Developer,IT,1978-03-26


4.	Вывести бренды, которые были куплены клиентами из сферы Financial Services, но не были куплены клиентами из сферы IT.

In [36]:
query_4 = """
WITH fs_brands AS (
    SELECT DISTINCT p.brand
    FROM customer c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN product p ON oi.product_id = p.product_id
    WHERE c.job_industry_category = 'Financial Services'
),
it_brands AS (
    SELECT DISTINCT p.brand
    FROM customer c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN product p ON oi.product_id = p.product_id
    WHERE c.job_industry_category = 'IT'
)
SELECT brand
FROM fs_brands
WHERE brand NOT IN (SELECT brand FROM it_brands)
ORDER BY brand
"""

result_4 = pd.read_sql(query_4, con)
result_4


/tmp/ipython-input-3366765528.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_4 = pd.read_sql(query_4, con)


,brand


5.	Вывести 10 клиентов (ID, имя, фамилия), которые совершили наибольшее количество онлайн-заказов (в штуках) брендов Giant Bicycles, Norco Bicycles, Trek Bicycles, при условии, что они активны и имеют оценку имущества (property_valuation) выше среднего среди клиентов из того же штата.

In [37]:
query_5 = """
WITH avg_valuation_by_state AS (
    SELECT state, AVG(property_valuation) AS avg_valuation
    FROM customer
    WHERE deceased_indicator = 'N'
    GROUP BY state
),
qualified_customers AS (
    SELECT c.customer_id, c.first_name, c.last_name, c.state, c.property_valuation
    FROM customer c
    JOIN avg_valuation_by_state a ON c.state = a.state
    WHERE c.deceased_indicator = 'N'
      AND c.property_valuation > a.avg_valuation
),
selected_orders AS (
    SELECT o.customer_id, o.order_id
    FROM orders o
    WHERE o.online_order = TRUE
      AND o.order_status = 'Approved'
),
selected_order_items AS (
    SELECT oi.order_id, oi.product_id
    FROM order_items oi
    JOIN product p ON oi.product_id = p.product_id
    WHERE p.brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
),
customer_order_counts AS (
    SELECT qc.customer_id, COUNT(DISTINCT so.order_id) AS online_order_count
    FROM qualified_customers qc
    JOIN selected_orders so ON qc.customer_id = so.customer_id
    JOIN selected_order_items soi ON so.order_id = soi.order_id
    GROUP BY qc.customer_id
)
SELECT qc.customer_id, qc.first_name, qc.last_name, coc.online_order_count
FROM qualified_customers qc
JOIN customer_order_counts coc ON qc.customer_id = coc.customer_id
ORDER BY coc.online_order_count DESC
LIMIT 10;
"""

result_5 = con.execute(query_5).fetchdf()
result_5

,customer_id,first_name,last_name,online_order_count
0,353,Antonia,Cardis,7
1,3221,Brigid,Quigley,7
2,534,Madel,Palffrey,6
3,714,Burtie,Scintsbury,6
4,25,Geoff,Assaf,6
5,1640,Erie,Worswick,6
6,1302,Ericka,Eggers,6
7,1117,Georgena,Guilaem,6
8,1,Laraine,Medendorp,6
9,787,Norma,Batrim,6


6.	Вывести всех клиентов (ID, имя, фамилия), у которых нет подтвержденных онлайн-заказов за последний год, но при этом они владеют автомобилем и их сегмент благосостояния не Mass Customer.

In [38]:
query_6 = """
WITH max_date AS (
    SELECT MAX(order_date) AS max_order_date FROM orders
),
last_year_orders AS (
    SELECT DISTINCT customer_id
    FROM orders, max_date
    WHERE online_order = TRUE
      AND order_status = 'Approved'
      AND order_date >= max_date.max_order_date - INTERVAL '1 year'
)
SELECT c.customer_id, c.first_name, c.last_name
FROM customer c
WHERE owns_car = 'Yes'
  AND wealth_segment != 'Mass Customer'
  AND c.customer_id NOT IN (SELECT customer_id FROM last_year_orders)
ORDER BY c.customer_id
"""

result_6 = pd.read_sql(query_6, con)
result_6




/tmp/ipython-input-2718151934.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_6 = pd.read_sql(query_6, con)


,customer_id,first_name,last_name
0,65,Yale,Tanser
1,71,Hoyt,Glavias
2,163,Kessia,Helder
3,184,Nappie,Paolo
4,191,Christopher,Heining
...,...,...,...
168,3976,Gretel,Chrystal
169,3989,Nicolas,Burdass
170,3992,Germain,Tireman
171,3997,Blanch,Nisuis


7.	Вывести всех клиентов из сферы 'IT' (ID, имя, фамилия), которые купили 2 из 5 продуктов с самой высокой list_price в продуктовой линейке Road.

In [39]:
query_7 = """
WITH top_5_road_products AS (
    SELECT product_id
    FROM product
    WHERE product_line = 'Road'
    ORDER BY list_price DESC
    LIMIT 5
),
it_customers AS (
    SELECT customer_id, first_name, last_name
    FROM customer
    WHERE job_industry_category = 'IT'
),
it_customer_purchases AS (
    SELECT DISTINCT ic.customer_id, oi.product_id
    FROM it_customers ic
    JOIN orders od ON ic.customer_id = od.customer_id
    JOIN order_items oi ON od.order_id = oi.order_id
    WHERE oi.product_id IN (SELECT product_id FROM top_5_road_products)
)
SELECT ic.customer_id, ic.first_name, ic.last_name
FROM it_customer_purchases c
JOIN it_customers ic ON c.customer_id = ic.customer_id
GROUP BY ic.customer_id, ic.first_name, ic.last_name
HAVING COUNT(DISTINCT c.product_id) >= 2
ORDER BY ic.customer_id
"""

result_7 = pd.read_sql(query_7, con)
result_7


/tmp/ipython-input-2093467634.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_7 = pd.read_sql(query_7, con)


,customer_id,first_name,last_name
0,604,Mella,Petrovsky
1,983,Shaylyn,Riggs
2,1683,Brenn,Bacon
3,2469,Kermie,Hedger
4,3406,Lucy,Lackmann


8.	Вывести клиентов (ID, имя, фамилия, сфера деятельности) из сфер IT или Health, которые совершили не менее 3 подтвержденных заказов в период 2017-01-01 по 2017-03-01, и при этом их общий доход от этих заказов превышает 10 000 долларов.
Разделить вывод на две группы (IT и Health) с помощью UNION.


In [40]:
query_8 = """
WITH confirmed_orders AS (
    SELECT o.customer_id, o.order_id
    FROM orders o
    WHERE o.order_status = 'Approved'
      AND o.order_date BETWEEN DATE '2017-01-01' AND DATE '2017-03-01'
),
order_revenue AS (
    SELECT co.customer_id, SUM(oi.item_list_price_at_sale) AS total_revenue, COUNT(DISTINCT co.order_id) AS order_count
    FROM confirmed_orders co
    JOIN order_items oi ON co.order_id = oi.order_id
    GROUP BY co.customer_id
),
it_customers AS (
    SELECT c.customer_id, c.first_name, c.last_name, c.job_industry_category
    FROM customer c
    WHERE c.job_industry_category = 'IT'
),
health_customers AS (
    SELECT c.customer_id, c.first_name, c.last_name, c.job_industry_category
    FROM customer c
    WHERE c.job_industry_category = 'Health'
),
it_result AS (
    SELECT ic.customer_id, ic.first_name, ic.last_name, ic.job_industry_category
    FROM it_customers ic
    JOIN order_revenue orv ON ic.customer_id = orv.customer_id
    WHERE orv.order_count >= 3 AND orv.total_revenue > 10000
),
health_result AS (
    SELECT hc.customer_id, hc.first_name, hc.last_name, hc.job_industry_category
    FROM health_customers hc
    JOIN order_revenue orv ON hc.customer_id = orv.customer_id
    WHERE orv.order_count >= 3 AND orv.total_revenue > 10000
)
SELECT * FROM it_result
UNION ALL
SELECT * FROM health_result
ORDER BY job_industry_category, customer_id
"""

result_8 = pd.read_sql(query_8, con)
result_8



/tmp/ipython-input-2768840670.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_8 = pd.read_sql(query_8, con)


,customer_id,first_name,last_name,job_industry_category
